In [71]:
import pandas as pd

In [72]:
# 讀取試算表
df = pd.read_excel('augment_test.xlsx')
for i in range(len(df)):
    if df['Similo result'][i] == "unknown":
        df.drop(i,axis=0, inplace=True)
        
# 設定顯示格式選項
pd.set_option('display.float_format', '{:.2f}'.format)  # 顯示小數位數為兩位

# 將DataFrame按照格式印出
print(df.to_string(index=False))

                                                                                                                                                                                Old xpath                                                                                                          New xpath (ground truth)                                                                                                                                                                                                                                    TAC xpath                                                                                                                                                                                                                                                                                                                            Similo xpath TAC result Similo result  TAC Sim SIM Sim
                                                                             

['Old xpath', 'New xpath (ground truth)', 'TAC xpath', 'Similo xpath',
       'TAC result', 'Similo result', 'TAC Sim', 'SIM Sim']

In [76]:
THRESHOLDS = [0,1,2,3,4,5,6,7] #測試每一個閾值
LEN = len(df) #全部df的長度
df_thresholds = [pd.DataFrame() for _ in range(len(THRESHOLDS))]
# 設定顯示格式選項
pd.set_option('display.float_format', '{:.2f}'.format)  # 顯示小數位數為兩位
for threshold in THRESHOLDS:
    print(f"threshold_{threshold}:")
    change_pos = 0
    change_neg = 0
    df_thresholds[threshold] = df.copy(deep=True)
    for index, row in df_thresholds[threshold].iterrows():
        # 檢查similarity有沒有低於threshold
        if row['SIM Sim'] < threshold:
            df_thresholds[threshold].at[index, "Similo result"] = "-" # 把找到的結果改成"-"
            if row['New xpath (ground truth)'] == "-": # 檢查正解是不是元素消失
                df_thresholds[threshold].at[index, "Similo result"] = "correct"
                change_pos+=1
            else: # 如果把對的改成錯的
                df_thresholds[threshold].at[index, "Similo result"] = "incorrect" 
                change_neg+=1
    incorrect = df_thresholds[threshold]["Similo result"].value_counts().values[0]
    correct = df_thresholds[threshold]["Similo result"].value_counts().values[1]
    print(f"pos: {change_pos}\tneg: {change_neg}")
    print(f"correct: {correct}\tincorrect: {incorrect}\t accuracy: {correct/LEN}")
    

threshold_0:
pos: 0	neg: 0
correct: 61	incorrect: 114	 accuracy: 0.3485714285714286
threshold_1:
pos: 0	neg: 0
correct: 61	incorrect: 114	 accuracy: 0.3485714285714286
threshold_2:
pos: 2	neg: 0
correct: 61	incorrect: 114	 accuracy: 0.3485714285714286
threshold_3:
pos: 6	neg: 1
correct: 65	incorrect: 110	 accuracy: 0.37142857142857144
threshold_4:
pos: 32	neg: 8
correct: 85	incorrect: 90	 accuracy: 0.4857142857142857
threshold_5:
pos: 78	neg: 20
correct: 49	incorrect: 126	 accuracy: 0.28
threshold_6:
pos: 88	neg: 34
correct: 48	incorrect: 127	 accuracy: 0.2742857142857143
threshold_7:
pos: 91	neg: 56
correct: 65	incorrect: 110	 accuracy: 0.37142857142857144


In [70]:
# 創建示例 DataFrame
df = pd.read_excel('augment_test.xlsx')

print(len(df))
# 刪除unknown的欄位
for i in range(len(df)):
    if df['Similo result'][i] == "unknown":
        df.drop(i,axis=0, inplace=True)


202
175


107
